### 학습목표
1. 로그데이터로 고객 이탈 페이지 확인하기

In [2]:
import pandas as pd
import numpy as np

#### 웹서버 로그 데이터
 - 웹서버에 클라이언트로의 요청(request) 전달 시, 해당 요청에 대한 정보(ip, 시각, 방문 페이지 등등)를 기록하는 파일
 - 기록되는 로그의 포맷(format)의 표준이 있으나 설정으로 포맷 변경 가능
 - 로그 데이터는 주로 웹 서버의 디버깅, 데이터 분석 등의 형태로 사용 됨
 - 예제에서 사용하는 형식
   - ip 세션아이디 사용자식별자 시각 요청 페이지 상태코드 바이트사이즈
   ```
   1.0.0.1 sessionid user59 [16/Dec/2019:02:00:08] GET /checkout 200 1508
   ```

In [3]:
logs = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/onlineretail/web.log', 
                   sep='\s',
                   engine='python',
                   names=['ip', 'session_id', 'user_id', 'datetime', 'request', 'url', 'status', 'bytesize'])

logs.head()

,ip,session_id,user_id,datetime,request,url,status,bytesize
0,4.5.4.5,69de169f-6eed-4e4d-ae5b-ff997b8c889f,user89,[01/Dec/2019T00:47:11],GET,/product_list,200,2107
1,4.5.4.5,69de169f-6eed-4e4d-ae5b-ff997b8c889f,user89,[01/Dec/2019T00:51:21],GET,/product_detail,200,1323
2,3.3.3.3.,3d46aad9-17eb-4af1-bc54-6ca91d7f8f6c,user2,[01/Dec/2019T00:51:43],GET,/product_list,200,2616
3,1.0.1.0,57623182-b78b-4bdc-b977-a2b34612c6d1,user45,[01/Dec/2019T01:04:02],GET,/product_list,200,2303
4,3.3.3.3.,3d46aad9-17eb-4af1-bc54-6ca91d7f8f6c,user2,[01/Dec/2019T01:12:28],GET,/product_detail,200,1830


In [4]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ip          1290 non-null   object
 1   session_id  1290 non-null   object
 2   user_id     1290 non-null   object
 3   datetime    1290 non-null   object
 4   request     1290 non-null   object
 5   url         1290 non-null   object
 6   status      1290 non-null   int64 
 7   bytesize    1290 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 80.8+ KB


In [5]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ip          1290 non-null   object
 1   session_id  1290 non-null   object
 2   user_id     1290 non-null   object
 3   datetime    1290 non-null   object
 4   request     1290 non-null   object
 5   url         1290 non-null   object
 6   status      1290 non-null   int64 
 7   bytesize    1290 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 80.8+ KB


#### 날짜 형식 변환

In [6]:
logs['datetime'] = logs['datetime'].apply(lambda date: date.replace('[','').replace(']',''))
logs['datetime'] = pd.to_datetime(logs['datetime'], format = '%d/%b/%YT%H:%M:%S')
logs.head()

,ip,session_id,user_id,datetime,request,url,status,bytesize
0,4.5.4.5,69de169f-6eed-4e4d-ae5b-ff997b8c889f,user89,2019-12-01 00:47:11,GET,/product_list,200,2107
1,4.5.4.5,69de169f-6eed-4e4d-ae5b-ff997b8c889f,user89,2019-12-01 00:51:21,GET,/product_detail,200,1323
2,3.3.3.3.,3d46aad9-17eb-4af1-bc54-6ca91d7f8f6c,user2,2019-12-01 00:51:43,GET,/product_list,200,2616
3,1.0.1.0,57623182-b78b-4bdc-b977-a2b34612c6d1,user45,2019-12-01 01:04:02,GET,/product_list,200,2303
4,3.3.3.3.,3d46aad9-17eb-4af1-bc54-6ca91d7f8f6c,user2,2019-12-01 01:12:28,GET,/product_detail,200,1830


#### 어떤 페이지에서 고객이 이탈을 할까?
 - 고객 이탈 페이지를 알면 해당 페이지를 분석하여 고객을 최종 단계로 더 많이 유도 가능
 - 대부분의 경우 다음 스텝으로 넘어갈때의 장벽이(신용카드 입력, 정보 입력 등등) 높은 경우가 해당 됨

In [24]:
logs.head()

,ip,session_id,user_id,datetime,request,url,status,bytesize
0,4.5.4.5,69de169f-6eed-4e4d-ae5b-ff997b8c889f,user89,2019-12-01 00:47:11,GET,/product_list,200,2107
1,4.5.4.5,69de169f-6eed-4e4d-ae5b-ff997b8c889f,user89,2019-12-01 00:51:21,GET,/product_detail,200,1323
2,3.3.3.3.,3d46aad9-17eb-4af1-bc54-6ca91d7f8f6c,user2,2019-12-01 00:51:43,GET,/product_list,200,2616
3,1.0.1.0,57623182-b78b-4bdc-b977-a2b34612c6d1,user45,2019-12-01 01:04:02,GET,/product_list,200,2303
4,3.3.3.3.,3d46aad9-17eb-4af1-bc54-6ca91d7f8f6c,user2,2019-12-01 01:12:28,GET,/product_detail,200,1830


In [ ]:
session_id, product_list, product_detail, cart, order_complete


#### 퍼널 스텝 dataframe 생성
 - 스텝 순서(ordering) 등을 명시하기 위해 사용

In [25]:
funnel_dict = {'/product_list' : 1, '/product_detail' : 2, '/cart' : 3, '/order_complete':4}
funnel_steps = pd.DataFrame.from_dict(funnel_dict, orient = 'index', columns = ['step_no'])
funnel_steps

,step_no
/product_list,1
/product_detail,2
/cart,3
/order_complete,4


#### session, url 로 grouping
 - user_id가 아닌 session을 기준으로 삼는 이유는 동일한 유저가 다른 세션으로 접속한 경우도 다른 경우로 간주해야 하기 때문
 - session_id와 url로 그루핑하여 가장 시간대가 빠른 해당 이벤트에 대해 추출

In [26]:
grouped = logs.groupby(['session_id', 'url'])['datetime'].agg(np.min)
grouped = pd.DataFrame(grouped).merge(funnel_steps, left_on='url', right_index = True)

grouped

,,datetime,step_no
session_id,url,,
000d99d8-d2d4-4e9a-bb06-69b1ae6442d9,/product_detail,2019-12-01 12:06:39,2
0155049d-32e7-44de-9b0d-4c02f63d6099,/product_detail,2019-12-04 00:22:44,2
020d4536-1341-4de1-87d3-e22ba8611af6,/product_detail,2019-12-19 06:25:48,2
0381411a-78d8-4c27-9622-3210b7ed62d6,/product_detail,2019-12-05 05:09:32,2
06268108-6228-4237-ac1d-7927dd44273d,/product_detail,2019-12-11 04:17:31,2
...,...,...,...
ed374836-99eb-4e31-8b0d-40e39d38bd54,/order_complete,2019-12-08 03:42:01,4
ef2c3b91-b701-4d46-85ac-96607f0fccc1,/order_complete,2019-12-16 05:48:56,4
f25e918d-f47e-4704-a923-19f1e106f618,/order_complete,2019-12-18 07:36:20,4


#### 퍼널 테이블 생성
 - 각 퍼널의 스텝이 순서대로 columns으로 오도록 변경

In [28]:
funnel = grouped.reset_index().pivot(index='session_id', columns = 'step_no', values = 'datetime')
funnel.columns = funnel_steps.index
funnel

,/product_list,/product_detail,/cart,/order_complete
session_id,,,,
000d99d8-d2d4-4e9a-bb06-69b1ae6442d9,2019-12-01 11:52:32,2019-12-01 12:06:39,NaT,NaT
0155049d-32e7-44de-9b0d-4c02f63d6099,2019-12-04 00:12:47,2019-12-04 00:22:44,NaT,NaT
020d4536-1341-4de1-87d3-e22ba8611af6,2019-12-19 06:22:54,2019-12-19 06:25:48,2019-12-19 06:58:23,NaT
0381411a-78d8-4c27-9622-3210b7ed62d6,2019-12-05 04:48:34,2019-12-05 05:09:32,2019-12-05 05:35:16,NaT
06268108-6228-4237-ac1d-7927dd44273d,2019-12-11 04:15:46,2019-12-11 04:17:31,2019-12-11 04:45:05,NaT
...,...,...,...,...
fc02c39b-5125-4036-8094-f16b7591854b,2019-12-14 04:06:41,NaT,NaT,NaT
fc3888e9-e8a2-4a05-a11a-0702d35160ea,2019-12-13 04:21:04,2019-12-13 04:55:29,2019-12-13 05:12:23,NaT
fcae3f0a-0500-4cb5-8b82-7c1d10d25beb,2019-12-10 08:11:34,2019-12-10 08:13:41,2019-12-10 08:40:43,NaT


#### 퍼널 카운트 계산
 - 각 퍼널 스텝별 카운트 계산

In [31]:
step_values = [funnel[index].notnull().sum() for index in funnel_steps.index]
step_values

[419, 351, 261, 84]

In [32]:
def show_funnel(funnel_values, funnel_steps):
  from plotly import graph_objects as go

  fig = go.Figure(go.Funnel(
      y = funnel_steps,
      x = funnel_values
  ))

  fig.show()

In [33]:
show_funnel(step_values, funnel_steps.index)

#### 평균 시간 계산
 - 각 퍼널별 소요 시간 계산

In [34]:
np.mean(funnel['/product_detail'] - funnel['/product_list'])

Timedelta('0 days 00:16:50.635327635')

In [35]:
np.mean(funnel['/cart'] - funnel['/product_detail'])

Timedelta('0 days 00:18:42.804597701')

In [36]:
np.mean(funnel['/order_complete'] - funnel['/cart'])

Timedelta('0 days 00:33:35.904761904')